In [9]:
import os
os.environ["OMP_NUM_THREADS"] = "1"

import sys 
sys.path.append("../../") 
from QC5.ansatze.ucc import *
from QC5.hamiltonians.FerOp_QubitOp import *
from QC5.algorithms.grad_ops_wrapper import *



import numpy as np
from pyscf import gto,scf,fci,cc,mcscf,ao2mo,mp

In [10]:
def genHn(n,Re):
    return [['H', 0., 0., i*Re] for i in range(n)]

# check RuCCSD

In [11]:
for n in [2,4]:
    print('**********Hn=',n)
    mol = gto.Mole()
    mol.atom = genHn(n,Re=0.74144)
    mol.basis = 'sto3g' 
    mol.charge = 0 
    mol.spin = n%2
    mol.symmetry = True
    mol.build()   
    
    # RHF
    mf = scf.RHF(mol)
    eRHF = mf.kernel()
    print('eRHF',eRHF)
    
    # classical information nelec and integrals
    nelec = mol.nelec
    norb = len(mf.get_occ())
    ecore = mol.energy_nuc()
    h1e = mf.get_hcore()
    h2e = mf._eri  
    mo_coeff = mf.mo_coeff 
    
    # RCCSD amplitudes 
    mycc = cc.CCSD(mf)
    ccsd = mycc.kernel()
    eCCSD = ccsd[0] + eRHF 
    t1,t2  = ccsd[1],ccsd[2]
    
    # FCI 
    molFCI = fci.FCI(mf)
    res = molFCI.kernel()
    eFCI = res[0]
    print('eFCI',res[0])
    
    # RHF basis
    fermion_transform = 'jordan_wigner'
    h1,h2 = get_ao2so_h1_h2_int(mo_coeff,h1e,h2e)
    qubit_hamiltonian = get_qubit_hamiltonian(ecore,h1,h2,fermion_transform)
    
    ucc = [RuCCSD,RuCCGSD1,RuCCGSD2,RuCCSDcomplex,RuCCGSD1complex,RuCCGSD2complex]
    for idx in range(len(ucc)):
        print('------------')
        UCCSD = ucc[idx](nelec, t1, t2, fermion_transform)
        UCCSD.parameter = 0.0 # set generalized amps=0

        ansatz = UCCSD.params_circuit
        init_amps = UCCSD.get_ccsd_initial_amps() # from pyscf CCSD
        
        print('check amps number:')
        print(np.isclose(len(ansatz.params_name),len(init_amps)))
        
        print('check pyscf CCSD as initial guess:')
        e0 = get_expect_value(qubit_hamiltonian,ansatz,init_amps)
        print(e0,np.isclose(e0,eCCSD),e0-eFCI)

**********Hn= 2
converged SCF energy = -1.11668219696281
eRHF -1.1166821969628056
E(CCSD) = -1.137269903217081  E_corr = -0.02058770625427591
eFCI -1.1372697371836944
------------
check amps number:
True
check pyscf CCSD as initial guess:
-1.1372693562980434 True 3.8088565101901395e-07
------------
check amps number:
True
check pyscf CCSD as initial guess:
-1.1372693562980434 True 3.8088565101901395e-07
------------
check amps number:
True
check pyscf CCSD as initial guess:
-1.137269356298046 True 3.808856483544787e-07
------------
check amps number:
True
check pyscf CCSD as initial guess:
-1.116859461756467 False 0.020410275427227287
------------
check amps number:
True
check pyscf CCSD as initial guess:
-1.116859461756467 False 0.020410275427227287
------------
check amps number:
True
check pyscf CCSD as initial guess:
-1.116859461756467 False 0.020410275427227287
**********Hn= 4
converged SCF energy = -2.09871650528698
eRHF -2.098716505286984
E(CCSD) = -2.139821107387557  E_corr = -

# check UuCCSD

In [12]:
for n in [3,6]:
    print('**********Hn=',n)
    mol = gto.Mole()
    mol.atom = genHn(n,Re=0.74144)
    mol.basis = 'sto3g' 
    mol.charge = 0 
    mol.spin = n%2
    mol.symmetry = True
    mol.build()   
    
    # UHF
    mf = scf.UHF(mol)
    eUHF = mf.kernel()
    print('eUHF',eUHF)
    
    # classical information nelec and integrals
    nelec = mol.nelec
    norb = len(mf.get_occ())
    ecore = mol.energy_nuc()
    h1e = mf.get_hcore()
    h2e = mf._eri  
    mo_coeff = mf.mo_coeff 
    
    # UCCSD amplitudes 
    mycc = cc.UCCSD(mf)
    ccsd = mycc.kernel()
    eCCSD = ccsd[0] + eUHF 
    t1,t2  = ccsd[1],ccsd[2]
    
    # FCI 
    molFCI = fci.FCI(mf)
    res = molFCI.kernel()
    eFCI = res[0]
    print('eFCI',res[0])
    
    # UHF basis
    fermion_transform = 'jordan_wigner'
    h1,h2 = get_ao2so_h1_h2_int(mo_coeff,h1e,h2e)
    qubit_hamiltonian = get_qubit_hamiltonian(ecore,h1,h2,fermion_transform)
    
    ucc = [UuCCSD,UuCCGSD1,UuCCGSD2,UuCCSDcomplex,UuCCGSD1complex,UuCCGSD2complex]
    for idx in range(len(ucc)):
        print('------------')
        UCCSD = ucc[idx](nelec, t1, t2, fermion_transform)
        UCCSD.parameter = 0.0 # set generalized amps=0

        ansatz = UCCSD.params_circuit
        init_amps = UCCSD.get_ccsd_initial_amps() # from pyscf CCSD
        
        print('check amps number:')
        print(np.isclose(len(ansatz.params_name),len(init_amps)))
        
        print('check pyscf CCSD as initial guess:')
        e0 = get_expect_value(qubit_hamiltonian,ansatz,init_amps)
        print(e0,np.isclose(e0,eCCSD),e0-eFCI)

**********Hn= 3
converged SCF energy = -1.50986304466201  <S^2> = 0.76515972  2S+1 = 2.0151027
eUHF -1.509863044662007
E(UCCSD) = -1.52696895838341  E_corr = -0.01710591372140292
eFCI -1.5269689776498312
------------
check amps number:
True
check pyscf CCSD as initial guess:
-1.526968716761819 True 2.608880123045054e-07
------------
check amps number:
True
check pyscf CCSD as initial guess:
-1.5269687167618202 True 2.608880109722378e-07
------------
check amps number:
True
check pyscf CCSD as initial guess:
-1.5269687167618744 True 2.608879567933542e-07
------------
check amps number:
True
check pyscf CCSD as initial guess:
-1.509961737136026 False 0.01700724051380509
------------
check amps number:
True
check pyscf CCSD as initial guess:
-1.5099617371360339 False 0.017007240513797317
------------
check amps number:
True
check pyscf CCSD as initial guess:
-1.5099617371361236 False 0.01700724051370761
**********Hn= 6
converged SCF energy = -3.08170730740195  <S^2> = 2.220446e-15  2S+1 =

# check Spinless ansatz

In [13]:
def resort_initial_amps(cir_params_name,initial_amplitudes):
    # for mindquantum 
    new_order = []
    for temp in cir_params_name:
        temp = temp.split('p',) # note the params_name is 'p'
        ordin = eval(temp[1])
        new_order.append(ordin)
    new_amps = numpy.zeros_like(initial_amplitudes)
    for idx,ordin in enumerate(new_order):
        new_amps[idx] = initial_amplitudes[ordin]       
    return new_amps 

In [16]:
fermion_transform = 'jordan_wigner'
from QC_master.hamiltonians.hamiltonian_heisenberg_model import *

In [17]:
for nsite in [4,8]:
    nelec = (nsite//4,nsite//4)
    print('-----nsite-----',nsite)
    UCCSD = NuCCGSD2(nsite, nelec, fermion_transform)
    ansatz = UCCSD.params_circuit
    qubit_hamiltonian = get_qubit_aniso_ham(nsite, 1, J=-1)
    nparams = UCCSD.get_amps_num()
    initial_amps = np.ones(nparams)
    amps = resort_initial_amps(ansatz.params_name,initial_amps)
    e0 = get_expect_value(qubit_hamiltonian, ansatz, amps)
    print(e0)

-----nsite----- 4


NameError: name 'numpy' is not defined

In [10]:
for nsite in [4,8]:
    print('-----nsite-----',nsite)
    nelec = (nsite//4,nsite//4)
    UCCSD = NuCCGSD2complex(nsite, nelec, fermion_transform)
    ansatz = UCCSD.params_circuit
    #print(len(ansatz.params_name),ansatz.params_name)
    qubit_hamiltonian = get_qubit_aniso_ham(nsite, 1, J=-1)
    nparams = UCCSD.get_amps_num()
    ones = np.ones(nparams//2).tolist() 
    zeros = np.zeros(nparams//2).tolist()
    initial_amps = np.concatenate((ones,zeros))
    amps = resort_initial_amps(ansatz.params_name,initial_amps)
    e0 = get_expect_value(qubit_hamiltonian, ansatz, amps)
    print(e0)

-----nsite----- 4
-1.497238543979185
-----nsite----- 8
-0.614194598993665
